In [95]:
# https://news.cnyes.com/news/cat/tw_stock_news

In [96]:
# 操作 browser 的 驅動程式
from selenium import webdriver

# 負責開啟和關閉 Chrome 的套件
from selenium.webdriver.chrome.service import Service

# 自動下載 Chrome Driver 的套件
from webdriver_manager.chrome import ChromeDriverManager

# 啟動瀏覽器工具的選項
from selenium.webdriver.chrome.options import Options

# 例外處理的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待、了解某個元素的狀態，通常與 exptected_conditions 和 By 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，經常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 隨機取得 User-Agent
from fake_useragent import UserAgent
# ua = UserAgent()

import pymysql
from pymysql.err import IntegrityError
from time import sleep
import json
import random
import os
from datetime import datetime
from datetime import timedelta
import re
from pprint import pprint


In [97]:
user_agent_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393",
    "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 6P Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36"
    "Mozilla/5.0 (iPhone14,3; U; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/19A346 Safari/602.1"
    # Add more user agent strings as needed
]
user_agent = random.choice(user_agent_list)

my_options = webdriver.ChromeOptions()
my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")  # 最大化視窗
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  # 設定為正體中文
my_options.add_argument('blink-settings=imagesEnabled=false')  # 不載入圖
my_options.add_argument(f'--user-agent={user_agent}')  # 隨機 user-angent 
my_options.add_experimental_option(
    "excludeSwitches", ['enable-automation', 'enable-logging'])  # 沒有異常log


driver = webdriver.Chrome(
    options=my_options,
    service=Service(ChromeDriverManager().install()))

In [98]:
# 建立下載路徑/資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
folderPath = os.path.join(os.getcwd(), 'crawler_data')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

#預設下載路徑
my_options.add_experimental_option("prefs", {
  "download.default_directory": folderPath
})

In [99]:

url = 'https://news.cnyes.com/news/cat/tw_stock_news'

listData = []
result = []
results = []
alinks = []

In [100]:
def visit():
    driver.get(url)


def scroll():

    innerHeight = 0
    offset = 0
    count = 0
    limit = 3

    while count <= limit:
        offset = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')

        # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(3)

        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )

        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1

        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if offset >= 500:
            break

def inputlink():
    links = driver.find_elements(
        By.CSS_SELECTOR, 
        "div > a._1Zdp"
    )
    times = driver.find_elements(
        By.CSS_SELECTOR, 
        "div > a._1Zdp > div > time"
    )
    
    ctime = datetime.now()
    today0am = ctime.replace(hour=0, minute=0, second=0, microsecond=0)
    yesterday0am = today0am - timedelta(days=1)


    # print("before",times[-1].text)
    # print(type(times[-1].text))
    try:
        while ":" in times[-1].text:
            scroll()

            sleep(4)
            
            times = driver.find_elements(
            By.CSS_SELECTOR, 
            "div > a._1Zdp > div > time"
            )

            links = driver.find_elements(
            By.CSS_SELECTOR, 
            "div > a._1Zdp"
            )

            check_links = driver.find_elements(
            By.CSS_SELECTOR,
            "div._2bFl.theme-list > div"
            )

            # print("after",times[-1].text)
        # sleep(2)
        
        # print(times[-1].text)
        # print(type(times[-1].text))
    except:
        pass
        

    
    for i in range(len(times)):
        article_datetime = times[i].get_attribute('innerText')
        # print(article_datetime)

        if ":" in article_datetime:
            time_hour = article_datetime.split(":")[0]
            time_mins = article_datetime.split(":")[1]
            post_time = ctime.replace(hour=int(time_hour), minute=int(time_mins), second=0, microsecond=0)

            href = links[i].get_attribute('href')
            alinks.append(href)
            
            # print(post_time)
        else: 
            pass
        

In [101]:
def craw(link):
    global result
    result = []

    # title
    title = driver.find_element(
        By.CSS_SELECTOR, '#content > div > div > div._2hZZ.theme-app.theme-newsdetail > main > div._uo1n._2l9x > h1').get_attribute('innerText')
    # time
    article_datetime = driver.find_element(
        By.XPATH, '//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/time').get_attribute('datetime')

    article_date_str = re.split('\+', article_datetime)[0].replace('T', ' ')
    # article_date_str = str(article_date_str)
    date_object = datetime.strptime(article_date_str, "%Y-%m-%d %H:%M:%S")
    article_date_str = str(date_object)

    # article
    article_list = driver.find_elements(
        By.CSS_SELECTOR, 'div > div._2E8y > p')
 
    article = ''
    for a in article_list:
        article += a.get_attribute("innerText")

    # reporter
    reporter = driver.find_element(
        By.CSS_SELECTOR, "span._3lKe > span").get_attribute("innerText").split(" ")[0][5:]
    
    # stocks
    stocks_list = driver.find_elements(
        By.CSS_SELECTOR, "main > div._1S0A > article > section._82F6 > nav > a > span")
    
    stocks = []
    for a in stocks_list:
        stocks.append(a.get_attribute("innerText"))
    stocks = str(stocks)

    result = {
        'news': '鉅亨網',
        'link': link, 
        'title': title, 
        'time': article_date_str, 
        'reporter': reporter, 
        'article': article,
        'stocks':stocks
        }
 
    # result = {
    #     'news': '工商時報',
    #     'link': link, 
    #     'title': title, 
    #     'time': article_date_str, 
    #     'reporter': reporter, 
    #     'article': article
    #     }
    
    # results.append(result)

    return result

In [102]:
if __name__ == "__main__":
    # 環境變數
    PORT = 3306
    HOST = "127.0.0.1"
    USER = "sunny"
    PASSWORD = "sunny"
    DATABASE = "news"
    TABLE = "news"

    # 資料庫連線
    connection = pymysql.connect(host=HOST, port=PORT, user=USER, 
                                 password=PASSWORD, database=DATABASE)
    cursor = connection.cursor()
    
    visit()
    inputlink()
    for link in alinks:
        driver.get(link)
        sql_result_dict = craw(link)
        sql_result_header = list(sql_result_dict.keys())
        print(sql_result_dict)
        # print(sql_result_dict[sql_result_header[0]])
        # print(sql_result_dict[sql_result_header[1]])
        # print(sql_result_dict[sql_result_header[3]])
        # print(type(sql_result_dict[sql_result_header[3]]))
        
        # 新增進資料庫
        # sql = 'INSERT INTO %s (news, link, title, time, reporter, article, stocks) VALUES ("%s", "%s", "%s", "%s", "%s", "%s");'\
        #                     % (TABLE, sql_result_dict[sql_result_header[0]], sql_result_dict[sql_result_header[1]], 
        #                        sql_result_dict[sql_result_header[2]], sql_result_dict[sql_result_header[3]],
        #                         sql_result_dict[sql_result_header[4]], sql_result_dict[sql_result_header[5]], 
        #                         sql_result_dict[sql_result_header[6]])
        
        sql = 'INSERT INTO {0} (news, link, title, reporter, article, stocks) VALUES ({1}, {2}, {3}, {4}, {5}, {6});'\
                            .format(TABLE, sql_result_dict[sql_result_header[0]], sql_result_dict[sql_result_header[1]], 
                               sql_result_dict[sql_result_header[2]],sql_result_dict[sql_result_header[3]],
                                sql_result_dict[sql_result_header[4]], sql_result_dict[sql_result_header[5]], 
                                sql_result_dict[sql_result_header[6]])

        # cursor.execute(sql)
        # connection.commit()

        sleep(2)
    connection.commit()
    connection.close()

    driver.quit() 




{'news': '鉅亨網', 'link': 'https://news.cnyes.com/news/id/5117036?exp=a', 'title': '〈普發6000來了〉雲品靠這波衝刺國旅淡季 Q1獲利有望再寫新猷', 'time': '2023-03-17 12:38:03', 'reporter': '王莞甯', 'article': '看好全民普發現金 6000 元的政策將激勵新一波消費動能，連鎖飯店餐飲業者雲品 (2748-TW) 今 (17) 日宣布看好這波商機，祭出每人每晚僅需 1500 元入住翰品酒店；法人指出，雲品同時還有布局館外委託事業，在宴會需求快速復甦下，住房和餐飲雙引擎挹注的獲利動能均值得期待，去年第四季獲利寫單季新高後，不排除第一季將再繳新紀錄，第二季相對淡季也樂觀。雲品指出，所屬雲朗觀光集團推出以 6000 元為單位的住房優惠吸客，分別有君品酒店、雲品溫泉酒店、翰品酒店新莊、翰品酒店高雄、翰品酒店花蓮、兆品酒店嘉義、兆品酒店台中和品文旅礁溪等 8 館，均在選項內。其中，以最優惠的翰品酒店系列來看，相當於每人每晚僅需 1500 元即可入住豪華套房等級的客房，讓住房享受更加升級。而最受國旅住客歡迎的雲品溫泉酒店，每人 6000 元即可入住 3 款指定房型。雲品去年第四季一舉大賺 1.19 億元，創單季獲利新高，每股純益 (EPS)1.31 元，反映宴會市場快速復甦、跨境旅遊市場回溫，再加上國旅持續暢旺。雲品說明，國旅和國際旅客住房市場正持續復甦，雲品酒店和君品酒店首季住房雙升可期。法人則指出，雲品在委託管理宴會餐飲市場上的長期布局，今年有望迎來明顯挹注，目前君品 Collection 和直營宴會廳總桌數已達 900 桌，隨著宴會疫後強勁復甦，全年接單有望寫新高。雲品正積極推展委託經營事業，目前共計 10 處，分別為翰品酒店新莊宴會廳、新北淡水嘉廬、桃園皇家薇庭、雲品酒店杭州、台東紅葉谷綠能溫泉園區、台中蘭克斯特、烏來紅河谷、台南市安平區、台北茹曦酒店餐飲與宴會和台中水湳圓觀，且持續有新案洽談中。法人認為，第一季是傳統年節餐飲旺季，緊接著第二季飯店業的相對淡季，又有 6000 元現金政策激勵，上半年雲品成長樂觀。', 'stocks': "['雲品', '餐飲', '宴會', '國旅', '觀光', '6000']"}
{'new

In [103]:
alinks
results

# connection.commit()
# connection.close()



[]